# Assignment 2

**Credits**: Andrea Galassi, Federico Ruggeri, Paolo Torroni

**Keywords**: Transformers, Question Answering, CoQA

## Overview

### Problem

Question Answering (QA) on [CoQA](https://stanfordnlp.github.io/coqa/) dataset: a conversational QA dataset.

### Task

Given a question $Q$, a text passage $P$, the task is to generate the answer $A$.<br>
$\rightarrow A$ can be: (i) a free-form text or (ii) unanswerable;

**Note**: an question $Q$ can refer to previous dialogue turns. <br>
$\rightarrow$ dialogue history $H$ may be a valuable input to provide the correct answer $A$.

### Models

We are going to experiment with transformer-based models to define the following models:

1.  $A = f_\theta(Q, P)$

2. $A = f_\theta(Q, P, H)$

where $f_\theta$ is the transformer-based model we have to define with $\theta$ parameters.

## The CoQA dataset

<center>
    <img src="https://drive.google.com/uc?export=view&id=16vrgyfoV42Z2AQX0QY7LHTfrgektEKKh" width="750"/>
</center>

For detailed information about the dataset, feel free to check the original [paper](https://arxiv.org/pdf/1808.07042.pdf).



## Rationales

Each QA pair is paired with a rationale $R$: it is a text span extracted from the given text passage $P$. <br>
$\rightarrow$ $R$ is not a requested output, but it can be used as an additional information at training time!

## Dataset Statistics

* **127k** QA pairs.
* **8k** conversations.
* **7** diverse domains: Children's Stories, Literature, Mid/High School Exams, News, Wikipedia, Reddit, Science.
* Average conversation length: **15 turns** (i.e., QA pairs).
* Almost **half** of CoQA questions refer back to **conversational history**.
* Only **train** and **validation** sets are available.

## Dataset snippet

The dataset is stored in JSON format. Each dialogue is represented as follows:

```
{
    "source": "mctest",
    "id": "3dr23u6we5exclen4th8uq9rb42tel",
    "filename": "mc160.test.41",
    "story": "Once upon a time, in a barn near a farm house, there lived a little white kitten named Cotton. 
    Cotton lived high up in a nice warm place above the barn where all of the farmer's horses slept. [...]" % <-- $P$
    "questions": [
        {
            "input_text": "What color was Cotton?",   % <-- $Q_1$
            "turn_id": 1
        },
        {
            "input_text": "Where did she live?",
            "turn_id": 2
        },
        [...]
    ],
    "answers": [
        {
            "span_start": 59,   % <-- $R_1$ start index
            "spand_end": 93,    % <-- $R_1$ end index
            "span_text": "a little white kitten named Cotton",   % <-- $R_1$
            "input_text" "white",   % <-- $A_1$      
            "turn_id": 1
        },
        [...]
    ]
}
```

### Simplifications

Each dialogue also contains an additional field ```additional_answers```. For simplicity, we **ignore** this field and only consider one groundtruth answer $A$ and text rationale $R$.

CoQA only contains 1.3% of unanswerable questions. For simplicity, we **ignore** those QA pairs.

In [1]:
import os
import os.path
import json
import typing
import urllib.request

import tensorflow.keras.callbacks
import tensorflow.keras.layers
import tensorflow.keras.losses
import tensorflow.keras.optimizers
import tensorflow.keras.regularizers
import numpy as np
import pandas as pd
import tensorflow as tf
import tqdm

## TASKS

### [Task 1] Remove unaswerable QA pairs - DONE 

Write your own script to remove unaswerable QA pairs from both train and validation sets. The unanswarable questions have -1 both in 'span_start' and 'span_end' and *unknown* in 'span_text'.      
***WARNING***: I found out that the sample in position 8246 has 'unknown' as answer therefore we must use the 'span_start' attribute to check for answerability.

### [Task 2] Train, Validation and Test splits - DONE

CoQA only provides a train and validation set since the test set is hidden for evaluation purposes.

We'll consider the provided validation set as a test set. <br>
$\rightarrow$ Write your own script to:
* Split the train data in train and validation splits (80% train and 20% val)
* Perform splits such that a dialogue appears in one split only! (i.e., split at dialogue level)
* Perform splitting using the following seed for reproducibility: 42

#### Reproducibility Memo

Check back tutorial 2 on how to fix a specific random seed for reproducibility!

### [Task 3] Model definition

Write your own script to define the following transformer-based models from [huggingface](https://HuggingFace.co/).

* [M1] DistilRoBERTa (distilberta-base)
* [M2] BERTTiny (bert-tiny)

**Note**: Remember to install the ```transformers``` python package!

**Note**: We consider small transformer models for computational reasons!

### [Task 4] Question generation with text passage $P$ and question $Q$

We want to define $f_\theta(P, Q)$. 

Write your own script to implement $f_\theta$ for each model: M1 and M2.

#### Formulation

Consider a dialogue on text passage $P$. 

For each question $Q_i$ at dialogue turn $i$, your model should take $P$ and $Q_i$ and generate $A_i$.

### [Task 5] Question generation with text passage $P$, question $Q$ and dialogue history $H$

We want to define $f_\theta(P, Q, H)$. Write your own script to implement $f_\theta$ for each model: M1 and M2.

#### Formulation

Consider a dialogue on text passage $P$. 

For each question $Q_i$ at dialogue turn $i$, your model should take $P$, $Q_i$, and $H = \{ Q_0, A_0, \dots, Q_{i-1}, A_{i-1} \}$ to generate $A_i$.

### [Task 6] Train and evaluate $f_\theta(P, Q)$ and $f_\theta(P, Q, H)$

Write your own script to train and evaluate your $f_\theta(P, Q)$ and $f_\theta(P, Q, H)$ models.

### Instructions

* Perform multiple train/evaluation seed runs: [42, 2022, 1337].$^1$
* Evaluate your models with the following metrics: SQUAD F1-score.$^2$
* Fine-tune each transformer-based models for **3 epochs**.
* Report evaluation SQUAD F1-score computed on the validation and test sets.

$^1$ Remember what we said about code reproducibility in Tutorial 2!

$^2$ You can use ```allennlp``` python package for a quick implementation of SQUAD F1-score: ```from allennlp_models.rc.tools import squad```. 

### [Task 7] Error Analysis

Perform a simple and short error analysis as follows:
* Group dialogues by ```source``` and report the worst 5 model errors for each source (w.r.t. SQUAD F1-score).
* Inspect observed results and try to provide some comments (e.g., do the models make errors when faced with a particular question type?)$^1$

$^1$ Check the [paper](https://arxiv.org/pdf/1808.07042.pdf) for some valuable information about question/answer types (e.g., Table 6, Table 8) 

## Dataset Download


In [2]:
import os
import urllib.request
from tqdm import tqdm

class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)
        
def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def download_data(data_path, url_path, suffix):    
    if not os.path.exists(data_path):
        os.makedirs(data_path)
        
    data_path = os.path.join(data_path, f'{suffix}.json')

    if not os.path.exists(data_path):
        print(f"Downloading CoQA {suffix} data split... (it may take a while)")
        download_url(url=url_path, output_path=data_path)
        print("Download completed!")

In [46]:
# Train data
train_url = "https://nlp.stanford.edu/data/coqa/coqa-train-v1.0.json"
download_data(data_path='coqa', url_path=train_url, suffix='train')

# Test data
test_url = "https://nlp.stanford.edu/data/coqa/coqa-dev-v1.0.json"
download_data(data_path='coqa', url_path=test_url, suffix='test')  # <-- Why test? See next slides for an answer!

## Create the dataframe and inspect the data

In [142]:
train_path = './coqa/train.json'
test_path = './coqa/test.json'

with open(train_path, 'r') as f:
    train_json = json.load(f)

with open(test_path, 'r') as f:
    test_json = json.load(f)


In [177]:
df_train = pd.json_normalize(train_json['data'])
df_test = pd.json_normalize(test_json['data'])
df_train.head()

,source,id,filename,story,questions,answers,name
0,wikipedia,3zotghdk5ibi9cex97fepx7jetpso7,Vatican_Library.txt,"The Vatican Apostolic Library (), more commonl...",[{'input_text': 'When was the Vat formally ope...,"[{'span_start': 151, 'span_end': 179, 'span_te...",Vatican_Library.txt
1,cnn,3wj1oxy92agboo5nlq4r7bndc3t8a8,cnn_fe05c61a7e48461f7883cdec387567029614f07b.s...,New York (CNN) -- More than 80 Michael Jackson...,"[{'input_text': 'Where was the Auction held?',...","[{'span_start': 243, 'span_end': 284, 'span_te...",cnn_fe05c61a7e48461f7883cdec387567029614f07b.s...
2,gutenberg,3bdcf01ogxu7zdn9vlrbf2rqzwplyf,data/gutenberg/txt/Zane Grey___Riders of the P...,"CHAPTER VII. THE DAUGHTER OF WITHERSTEEN \n\n""...",[{'input_text': 'What did Venters call Lassite...,"[{'span_start': 841, 'span_end': 880, 'span_te...",data/gutenberg/txt/Zane Grey___Riders of the P...
3,cnn,3ewijtffvo7wwchw6rtyaf7mfwte0p,cnn_0c518067e0df811501e46b2e1cd1ce511f1645b7.s...,(CNN) -- The longest-running holiday special s...,"[{'input_text': 'Who is Rudolph's father?', 't...","[{'span_start': 500, 'span_end': 557, 'span_te...",cnn_0c518067e0df811501e46b2e1cd1ce511f1645b7.s...
4,gutenberg,3urfvvm165iantk80llvkwwbjs7uzh,data/gutenberg/txt/Rafael Sabatini___Love-at-A...,CHAPTER XXIV. THE INTERRUPTED MASS \n\nThe mor...,"[{'input_text': 'Who arrived at the church?', ...","[{'span_start': 254, 'span_end': 297, 'span_te...",data/gutenberg/txt/Rafael Sabatini___Love-at-A...


In the dataset above we have some features that are not useful for our task therefore we decided to remove them.

In [178]:
features_to_remove = ['source', 'id', 'filename', 'name']
features_to_remove_test = features_to_remove + [f'additional_answers.{i}' for i in range(3)]

df_train.drop(features_to_remove, axis=1, inplace=True)
df_test.drop(features_to_remove_test, axis=1, inplace=True)

In [145]:
df_train.head()

,story,questions,answers
0,"The Vatican Apostolic Library (), more commonl...",[{'input_text': 'When was the Vat formally ope...,"[{'span_start': 151, 'span_end': 179, 'span_te..."
1,New York (CNN) -- More than 80 Michael Jackson...,"[{'input_text': 'Where was the Auction held?',...","[{'span_start': 243, 'span_end': 284, 'span_te..."
2,"CHAPTER VII. THE DAUGHTER OF WITHERSTEEN \n\n""...",[{'input_text': 'What did Venters call Lassite...,"[{'span_start': 841, 'span_end': 880, 'span_te..."
3,(CNN) -- The longest-running holiday special s...,"[{'input_text': 'Who is Rudolph's father?', 't...","[{'span_start': 500, 'span_end': 557, 'span_te..."
4,CHAPTER XXIV. THE INTERRUPTED MASS \n\nThe mor...,"[{'input_text': 'Who arrived at the church?', ...","[{'span_start': 254, 'span_end': 297, 'span_te..."


At this point we can split the dataset in training and validation and then we separate the story and the questions from the answers for readability.

In [179]:
rand_seed = 42

def split_train_validation(df: pd.DataFrame, val_size: float=0.2, 
    random_state: int=None) -> typing.Tuple[pd.DataFrame, pd.DataFrame]:
    '''
        It takes as input a dataframe and it splits it in 2 dataframes considering the 
        'val_size' parameter. 
    '''

    validation = df.sample(frac=val_size, random_state=random_state) 
    training = df.drop(validation.index).sample(frac=1, random_state=random_state)

    return training.reset_index(drop=True), validation.reset_index(drop=True)

In [180]:
df_train, df_val = split_train_validation(df_train, 0.2, rand_seed)
print(f"The dimensions of the datasets are:\n"
     f"Training dataset -> {df_train.shape} - Validation dataset -> {df_val.shape}")

The dimensions of the datasets are:
Training dataset -> (5759, 3) - Validation dataset -> (1440, 3)


In [181]:
def split_dataset(df: pd.DataFrame, to_drop: typing.List[str]=[]) -> typing.Tuple[pd.DataFrame, pd.DataFrame]:
    '''
        Given a Pandas dataframe this function returns 2 dataframe, the first one
        contains the story and the questions (X) and the second one contains the answers (y).
    '''
    questions = [{'story':df.story[i], **quest} for i, lis in enumerate(df.questions) for quest in lis]
    answers = [ans for lis in df.answers for ans in lis]

    X = pd.DataFrame.from_dict(questions, orient="columns").drop(to_drop, axis=1)
    y = pd.DataFrame.from_dict(answers, orient="columns").drop(to_drop, axis=1)
    
    return X, y

In [182]:
x_train, y_train = split_dataset(df_train, to_drop=['turn_id', 'bad_turn'])
x_val, y_val = split_dataset(df_val, to_drop=['turn_id', 'bad_turn'])
x_test, y_test = split_dataset(df_test, to_drop=['turn_id'])

print(f"The training set has {len(x_train)} samples, the validation set has {len(x_val)} samples"
      f"and the test set has {len(x_test)} samples.")

The training set has 86898 samples, the validation set has 21749 samplesand the test set has 7983 samples.


Now we will remove all the unanswerable questions as written in the assignment.

In [183]:
keep_rows_train = list(y_train.span_start > 0)
keep_rows_val = list(y_val.span_start > 0)
keep_rows_test = list(y_test.span_start > 0)

x_train = x_train.iloc[keep_rows_train].reset_index(drop=True)
y_train = y_train.iloc[keep_rows_train].reset_index(drop=True)

x_val = x_val.iloc[keep_rows_val].reset_index(drop=True)
y_val = y_val.iloc[keep_rows_val].reset_index(drop=True)

x_test = x_test.iloc[keep_rows_test].reset_index(drop=True)
y_test = y_test.iloc[keep_rows_test].reset_index(drop=True)

print(f"We have removed:\n- {len(x_train) - len(keep_rows_train)} unanswerable questions from the training set;\n"
      f"- {len(x_val) - len(keep_rows_val)} unanswerable questions from the validation set;\n"
      f"- {len(x_test) - len(keep_rows_test)} unanswerable questions from the test set;")

We have removed:
- -3490 unanswerable questions from the training set;
- -900 unanswerable questions from the validation set;
- -288 unanswerable questions from the test set;


In [184]:
print(f"The shape of the questions dataset is {x_train.shape}")
x_train.head()

The shape of the questions dataset is (83408, 2)


,story,input_text
0,CHAPTER 52 \n\nNicholas despairs of rescuing M...,Who is losing their nerve?
1,CHAPTER 52 \n\nNicholas despairs of rescuing M...,Who is he worried about?
2,CHAPTER 52 \n\nNicholas despairs of rescuing M...,What is her last name?
3,CHAPTER 52 \n\nNicholas despairs of rescuing M...,Does he get it back?
4,CHAPTER 52 \n\nNicholas despairs of rescuing M...,Who is an obstacle for him?


In [185]:
print(f"The shape of the answers dataset is {y_train.shape}")
y_train.head()

The shape of the answers dataset is (83408, 4)


,span_start,span_end,span_text,input_text
0,13,22,Nicholas,Nicholas is
1,43,51,Madeline,Madeline
2,52,56,Bray,Bray
3,62,89,plucks up his Spirits again,Yes
4,190,197,Newman,Newman


# Assignment Evaluation

The following assignment points will be awarded for each task as follows:

* Task 1, Pre-processing $\rightarrow$ 0.5 points.
* Task 2, Dataset Splitting $\rightarrow$ 0.5 points.
* Task 3 and 4, Models Definition $\rightarrow$ 1.0 points.
* Task 5 and 6, Models Training and Evaluation $\rightarrow$ 2.0 points.
* Task 7, Analysis $\rightarrow$ 1.0 points.
* Report $\rightarrow$ 1.0 points.

**Total** = 6 points <br>

We may award an additional 0.5 points for outstanding submissions. 
 
**Speed Bonus** = 0.5 extra points <br>

# Report

We apply the rules described in Assignment 1 regarding the report.
* Write a clear and concise report following the given overleaf template (**max 2 pages**).
* Report validation and test results in a table.$^1$
* **Avoid reporting** code snippets or copy-paste terminal outputs $\rightarrow$ **Provide a clean schema** of what you want to show

# Comments and Organization

Remember to properly comment your code (it is not necessary to comment each single line) and don't forget to describe your work!

Structure your code for readability and maintenance. If you work with Colab, use sections. 

This allows you to build clean and modular code, as well as easy to read and to debug (notebooks can be quite tricky time to time).

# FAQ (READ THIS!)

---

**Question**: Does Task 3 also include data tokenization and conversion step?

**Answer:** Yes! These steps are usually straightforward since ```transformers``` also offers a specific tokenizer for each model.

**Example**: 

```
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
encoded_text = tokenizer(text)
%% Alternatively
inputs = tokenizer.tokenize(text, add_special_tokens=True, max_length=min(max_length, 512))
input_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']
```

**Suggestion**: Hugginface's documentation is full of tutorials and user-friendly APIs.

---
---

**Question**: I'm hitting **out of memory error** when training my models, do you have any suggestions?

**Answer**: Here are some common workarounds:

1. Try decreasing the mini-batch size
2. Try applying a different padding strategy (if you are applying padding): e.g. use quantiles instead of maximum sequence length

---
---

# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Andrea Galassi -> a.galassi@unibo.it
* Federico Ruggeri -> federico.ruggeri6@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

# The End!

Questions?